In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [25]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model

mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')

In [26]:
# modify models
# load blood exchange bounds
import pandas as pd
df = pd.read_excel(Path().cwd() / 'rxn_bounds/blood_selection.xlsx') # with col names 'id' 'lb' 'ub'

# set objective function (PR ATP DM)
mod.objective = 'MAR03964'

In [28]:
# ATP yield per metabolite (flux of 1 for each met)
def ATP_yield_analysis_per_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (-1,1000) # open reaction
                m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
                fba = m.optimize()
                results_dict[row['id_single']] = fba.objective_value
                m.reactions.get_by_id(row['id_single']).bounds = (0,1000) # close reaction again
    return results_dict

r1 = ATP_yield_analysis_per_aa(mod,(0,1000)) # anearobic
r2 = ATP_yield_analysis_per_aa(mod,(-2.2,1000)) # limited oxygen
r3 = ATP_yield_analysis_per_aa(mod,(-1000,1000)) # unlimited oxyen

df_results = pd.DataFrame([r1.keys(),r1.values(),r2.values(),r3.values()]).transpose()
df_results.to_clipboard()
df_results

,0,1,2,3
0,MAR09034,2.0,13.138325,31.5
1,MAR09048,0.0,0.0,0.0
2,MAR09135,0.0,11.013325,14.75
3,MAR09047,0.0,0.0,0.0
4,MAR09063,0.522727,11.182626,22.0
5,MAR09061,0.0,11.013325,14.75
6,MAR09067,0.090909,7.75,7.75
7,MAR09041,0.0,10.717467,33.25
8,MAR09046,0.0,10.694739,28.5
9,MAR09068,0.0,10.558376,26.0


In [30]:
# ATP yield met exchange allowed (including glucose)
def ATP_yield_analysis(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (row['lb'],row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis(mod, (0,1000)))
print(ATP_yield_analysis(mod, (-2.2,1000)))
print(ATP_yield_analysis(mod, (-1000,1000)))

5.833096967283177
16.857007721427323
53.573534010213145


In [31]:
# ATP yield only glucose exchange allowed
def ATP_yield_analysis_only_glucose(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        m.reactions.get_by_id('MAR09034').bounds = (-2.622342019,1000) # glucose
        m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_only_glucose(mod, (0,1000)))
print(ATP_yield_analysis_only_glucose(mod, (-2.2,1000)))
print(ATP_yield_analysis_only_glucose(mod, (-1000,1000)))

5.244684037999971
16.41219672835558
82.6037735985016


In [33]:
# ATP yield met exchange allowed, but no glucose allowed
def ATP_yield_analysis_no_glucose(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (row['lb'],row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034').bounds = (0,1000) # glucose
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose(mod, (0,1000)))
print(ATP_yield_analysis_no_glucose(mod, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose(mod, (-1000,1000)))

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


None


C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


None
None


C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [39]:
# ATP yield met exchange allowed (high!), but no glucose allowed
def ATP_yield_analysis_no_glucose_high_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (-1,row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034').bounds = (0,1000) # glucose
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose_high_aa(mod, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod, (-1000,1000)))

6.833333333333826
18.000846023688688
509.31670808188505


In [38]:
# ATP yield per metabolite (low metabolite influx)
def ATP_yield_analysis_per_aa_low_influx(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (row['lb'],row['ub']) # open reaction
                m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
                fba = m.optimize()
                results_dict[row['id_single']] = fba.objective_value
                m.reactions.get_by_id(row['id_single']).bounds = (0,1000) # close reaction again
    return results_dict

r1 = ATP_yield_analysis_per_aa_low_influx(mod,(0,1000)) # anearobic
r2 = ATP_yield_analysis_per_aa_low_influx(mod,(-2.2,1000)) # limited oxygen
r3 = ATP_yield_analysis_per_aa_low_influx(mod,(-1000,1000)) # unlimited oxyen


df_results2 = pd.DataFrame([r1.keys(),r1.values(),r2.values(),r3.values()]).transpose()
df_results2.to_clipboard()
df_results2

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


,0,1,2,3
0,MAR09034,5.244684,16.412197,82.603774
1,MAR09048,0.0,0.0,0.0
2,MAR09135,None,None,None
3,MAR09047,0.0,0.0,0.0
4,MAR09063,0.17297,7.279766,7.279766
5,MAR09061,0.0,3.514808,3.514808
6,MAR09067,0.013244,1.129064,1.129064
7,MAR09041,0.0,4.43099,4.43099
8,MAR09046,0.0,3.749712,3.749712
9,MAR09068,0.0,2.789485,2.789485


In [35]:
# ATP yield, no glucose, low aa exchange
def ATP_yield_analysis_per_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (row['lb'],row['ub']) # open reaction
                m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034').bounds = (0,1000) # glucose bounds
        
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_per_aa(mod,(0,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod,(-2.2,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod,(-1000,1000))) # unlimited oxyen

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


None


C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


None


C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


None


KeyError: 'MAR09034'

In [41]:
# ATP  no glucose, mets allowed, lactate -1
def ATP_yield_analysis_no_glucose_high_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (row['lb'],row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034').bounds = (0,1000) # glucose
        m.reactions.get_by_id('MAR09135').bounds = (-1,1000) # LACTATE # MAR09135_RPE
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose_high_aa(mod, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod, (-1000,1000)))


0.5884129301457758
11.649775017024883
52.74810723573272


In [42]:
# ATP  no glucose, mets allowed, lactate -1
def ATP_yield_analysis_no_glucose_high_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (-1,row['ub']) # open blood exchange reactions
        m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034').bounds = (0,1000) # glucose
        m.reactions.get_by_id('MAR09135').bounds = (4.544294699,1000) # LACTATE # MAR09135_RPE
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_no_glucose_high_aa(mod, (0,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod, (-2.2,1000)))
print(ATP_yield_analysis_no_glucose_high_aa(mod, (-1000,1000)))


5.858910369620675
16.632671965979853
427.53836127163567


In [46]:
# ATP yield, no glucose, high aa exchange
def ATP_yield_analysis_per_aa(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]    
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (-1,1000) # open reaction
                m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
        m.reactions.get_by_id('MAR09034').bounds = (0,1000) # glucose bounds
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_per_aa(mod,(0,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod,(-2.2,1000))) # unlimited oxyen
print(ATP_yield_analysis_per_aa(mod,(-1000,1000))) # unlimited oxyen


6.833333333333334
18.000846023688663
509.3167080818838


In [51]:
# TEST
def ATP_yield_analysis_only_glucose(mod, oxygen_bounds):
    results_dict = dict()
    # close PR exchange reactions, open RPE exchange upper boundaries
    from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
    models = [mod]
    for m in models:
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,0)
        rxns = [r for r in m.reactions if len(r.products) == 0]
        for r in rxns:
            r.bounds=(0,1000)
        for index, row in df.iterrows():  
            if row.id_single in [r.id for r in m.reactions]:
                m.reactions.get_by_id(row['id_single']).bounds = (-1,row['ub']) # blood exchange
                fba = m.optimize()
        m.reactions.get_by_id('MAR09135').bounds = (-1,1000) # LACTATE 
        m.reactions.get_by_id('MAR09034').bounds = (-1,1000) # glucose
        m.reactions.get_by_id('MAR09048').bounds = oxygen_bounds # OXYGEN
        fba = m.optimize()
    return fba.objective_value

print(ATP_yield_analysis_only_glucose(mod, (0,1000)))
print(ATP_yield_analysis_only_glucose(mod, (-2.2,1000)))
print(ATP_yield_analysis_only_glucose(mod, (-1000,1000)))

8.833333333340363
20.000846023688723
540.8167080818838


In [37]:
from src.modify_model import close_PR_EX, open_RPE_EX_ub, close_EX
m = [mod]
rxns = [r for r in m.reactions if len(r.products) == 0]
for r in rxns:
    r.bounds=(0,1000)
m.reactions.get_by_id('MAR09071_RPE').bounds = (-0.024450347,1000) # glutamic acid
m.reactions.get_by_id('MAR09048').bounds = (-1000,1000) # OXYGEN
m.reactions.get_by_id('MAR09034').bounds = (0,1000) # glucose bounds
fba = m.optimize()
print(fba.objective_value)

AttributeError: 'list' object has no attribute 'reactions'

In [37]:
m = mod
m = close_EX(m)
m = open_RPE_EX_ub(m)
for index, row in df.iterrows():
    m.reactions.get_by_id(row['id_single']).lower_bound=row['lb']
m.reactions.get_by_id('MAR09034').lower_bound=0

Reaction identifier,MAR09034_RPE
Name,
Memory address,0x23384c57ac0
Stoichiometry,MAM01965e_RPE --> glucose -->
GPR,
Lower bound,0
Upper bound,1000


In [20]:
l= ['MAR09034',
  'MAR09135_RPE',
  'MAR09063_RPE',
  'MAR09061_RPE',
  'MAR09067_RPE',
  'MAR09041_RPE',
  'MAR09046_RPE',
  'MAR09068_RPE',
  'MAR09044_RPE',
  'MAR09069_RPE',
  'MAR09040_RPE',
  'MAR09038_RPE',
  'MAR09066_RPE',
  'MAR09064_RPE',
  'MAR09043_RPE',
  'MAR09065_RPE',
  'MAR09045_RPE',
  'MAR09062_RPE',
  'MAR09042_RPE',
  'MAR09070_RPE',
  'MAR11961_RPE',
  'MAR09039_RPE']

for 

In [ ]:
from src.analysis import fba_analysis, create_permutation_dicts

# set simulation parameters
# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
# 'MAR09048' O2[e_RPE] <=> 
# 'MAR04896_PR_RPE' O2[c_PR] <=> O2[e_RPE_PR]
#  MAR03964_RPE RPE ATP hydrolysis
#  MAR09089_RPE    phospholipids extracellular pool[e_RPE] -->
#  MAR13037_RPE    fatty acid-VLDL pool[e_RPE] -->
#  MAR13039_RPE    fatty acid-uptake pool[e_RPE] -->
#  MAR09209_RPE    fatty acid pool[e_RPE] -->
Human1_dict = create_permutation_dicts\
({'MAR09034':[(0,1000),(-2.2,1000),(-1000,1000)],\
  'MAR09135_RPE':[(-1000,1000)],\
 'MAR09089_RPE':[(0,1000),(-1,1000)],\
  'MAR13037_RPE':[(0,1000),(-1,1000)],\
  'MAR13039_RPE':[(0,1000),(-1,1000)],\
  'MAR09209_RPE':[(0,1000),(-1,1000)]})

 {'MAR09034': (0, 1000),
  'MAR09135_RPE': (0, 1000),
  'MAR09063_RPE': (0, 1000),
  'MAR09061_RPE': (0, 1000),
  'MAR09067_RPE': (0, 1000),
  'MAR09041_RPE': (0, 1000),
  'MAR09046_RPE': (0, 1000),
  'MAR09068_RPE': (0, 1000),
  'MAR09044_RPE': (0, 1000),
  'MAR09069_RPE': (0, 1000),
  'MAR09040_RPE': (0, 1000),
  'MAR09038_RPE': (0, 1000),
  'MAR09066_RPE': (0, 1000),
  'MAR09064_RPE': (0, 1000),
  'MAR09043_RPE': (0, 1000),
  'MAR09065_RPE': (0, 1000),
  'MAR09045_RPE': (0, 1000),
  'MAR09062_RPE': (0, 1000),
  'MAR09042_RPE': (0, 1000),
  'MAR09070_RPE': (0, 1000),
  'MAR11961_RPE': (0, 1000),
  'MAR09039_RPE': (0, 1000)}


 ['MAR09034': (0, 1000),
  'MAR09135_RPE': (0, 1000),
  'MAR09063_RPE': (0, 1000),
  'MAR09061_RPE': (0, 1000),
  'MAR09067_RPE': (0, 1000),
  'MAR09041_RPE': (0, 1000),
  'MAR09046_RPE': (0, 1000),
  'MAR09068_RPE': (0, 1000),
  'MAR09044_RPE': (0, 1000),
  'MAR09069_RPE': (0, 1000),
  'MAR09040_RPE': (0, 1000),
  'MAR09038_RPE': (0, 1000),
  'MAR09066_RPE': (0, 1000),
  'MAR09064_RPE': (0, 1000),
  'MAR09043_RPE': (0, 1000),
  'MAR09065_RPE': (0, 1000),
  'MAR09045_RPE': (0, 1000),
  'MAR09062_RPE': (0, 1000),
  'MAR09042_RPE': (0, 1000),
  'MAR09070_RPE': (0, 1000),
  'MAR11961_RPE': (0, 1000),
  'MAR09039_RPE': (0, 1000)}




In [ ]:
from src.analysis import fba_analysis, create_permutation_dicts                                
fba1 = fba_analysis(mod,lipids_exchange_dict,'MAR03964_PR',)
fba2 = fba_analysis(mod_Human1_Human1,lipids_exchange_dict,'MAR03964_PR')

In [ ]:
def fba_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model.copy() as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            pf[i] = p_fba.to_frame()['fluxes']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    # select internal fluces only
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion


In [ ]:
    boundary_dicts = Human1_dict_p
    objective = 'MAR03964_PR'
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with mod as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            p_fba = pfba(m) # pfba
            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            pf[i] = p_fba.to_frame()['fluxes']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]

In [ ]:
from src.get_info import make_rxn_df
df = make_rxn_df(mod)
DF2 = df[df.index.isin([r.id for r in mod.reactions if len(r.products)==0])]
DF2.to_clipboard()

In [ ]:
for m in [mod, mod_Human1_Human1]:
    m = close_PR_EX(m)
    m = open_RPE_EX_ub(m)
    m.reactions.get_by_id('MAR09048').bounds=(-1000,1000)
    m.reactions.get_by_id('MAR09034').bounds=(-10,1000)
    m.reactions.get_by_id('MAR03964_RPE').bounds = (40,40)
    m.objective = 'MAR03964_PR'

In [ ]:
mod_Human1_Human1.optimize()

In [ ]:
mod.optimize()

In [ ]:
mod_old.optimize()

In [ ]:
mod.reactions.get_by_id('MAR10023_RPE_PR')

In [ ]:
from cobra.io import write_sbml_model
# save combined RPE_PR models
write_sbml_model(mod, "mod.xml")

In [ ]:
# set simulation parameters
from src.analysis import create_permutation_dicts

# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
Human1_dict = {'MAR09048':[(-10,1000)],'MAR09034':[(-30, 1000)],'MAR03964_RPE':[(0,0),(40,40),(80,80)]}
Recon3D_dict = {'EX_o2s[e]_RPE':[(-10,1000)],'EX_glc_D[e]_RPE':[(-30,1000)],'DM_atp_c__RPE':[(0,0),(40,40),(80,80)]}
mitocore_dict = {'EX_o2_e':[(-1000,1000)],'EX_glc_D_e':[(-300,1000)]}

Human1_dict_p = create_permutation_dicts(Human1_dict)
Recon3D_dict_p = create_permutation_dicts(Recon3D_dict)
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

In [ ]:
from src.modify_model import change_bounds
mod.reactions.get_by_id('MAR09034').bounds = (-10,1000)
mod.reactions.get_by_id('MAR03964_RPE').bounds = (-100,1000)

In [ ]:
df = make_rxn_df(mod)

In [ ]:
def fba_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f_min), pd.DataFrame(f_max),pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["min","max", "parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion

In [ ]:
def fba_fva_analysis(model, boundary_dicts, objective):
    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from cobra.flux_analysis import flux_variability_analysis, pfba
    from src.analysis import FVA_FBA_analysis
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    ovs = dict() # objective values
    f = dict() # fba fluxes
    pf = dict() # pfba fluxes
    f_min = dict() # fva min
    f_max = dict() # fva max
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 1
        
    with model as m:
        # set objective
        m.objective = objective
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            p_fba = pfba(m) # pfba
            fva = flux_variability_analysis(m, loopless=False) #fva

            # write analysis info / results into dicts
            
            # model bounds
            bounds[i] = pd.DataFrame(d, index = ['lb', 'ub']).T    
            # objective values
            ovs[i] = pd.DataFrame([objective, fba.objective_value], columns = [m.objective.direction], index = ['ID', 'value']).T   
            # uptake / secretion
            uptake_summary = m.summary().uptake_flux
            secretion_summary = m.summary().secretion_flux
            uptake[i] = pd.DataFrame(uptake_summary['flux'])
            secretion[i] = pd.DataFrame(secretion_summary['flux'])
            # all fluxes
            f[i] = fba.to_frame()['fluxes']
            pf[i] = p_fba.to_frame()['fluxes']
            f_min[i] = fva['minimum']
            f_max[i] = fva['minimum']
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)   
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    bounds_df = pd.concat(bounds)
    ovs_df = pd.concat(ovs)
    
    # uptake / secretion dfs
    uptake_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in uptake_summary['metabolite']],\
             index=uptake_summary.index,columns=['met_id','met_name'])
    secretion_mets = pd.DataFrame([[met,m.metabolites.get_by_id(met).name] for met in secretion_summary['metabolite']],\
             index=secretion_summary['metabolite'].index,columns=['met_id','met_name'])
    uptake_df = pd.merge(uptake_mets, pd.concat(uptake,axis=1), left_index=True, right_index=True)
    secretion_df = pd.merge(secretion_mets, pd.concat(secretion,axis=1), left_index=True, right_index=True)   
    # sort uptake / secretion dfs
    uptake_df = uptake_df.sort_values(by=[c for c in uptake_df.columns if 'flux' in c],ignore_index=True,ascending=False)
    secretion_df = secretion_df.sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ignore_index=True,ascending=True)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f_min), pd.DataFrame(f_max),pd.DataFrame(pf), pd.DataFrame(f)],\
          keys=["min","max", "parsimonious flux","flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    
    
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        bounds_df.to_excel(writer, sheet_name = 'altered_bounds')   
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')   
    return bounds, ovs, f, pf, f_min, f_max, uptake, secretion

In [ ]:
mitocore_dict = {'EX_o2_e':[(-10,1000),(-1000,1000)],'EX_glc_D_e':[(-1,1000),(-30,1000),(-40,1000)]} # allow oxygen and glucose in
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

bounds, ovs, f, pf, f_min, f_max, uptake, secretion = fba_fva_analysis(mito, mitocore_dict_p,'OF_ATP_MitoCore')

In [ ]:
mitocore_dict = {'EX_o2_e':[(-10,1000),(-1000,1000)],'EX_glc_D_e':[(-1,1000),(-30,1000)]} # allow oxygen and glucose in
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

bounds, ovs, f, pf, f_min, f_max = fba_fva_analysis(mito, mitocore_dict_p,'OF_ATP_MitoCore')

In [ ]:
Human1_dict = create_permutation_dicts({'MAR09048':[(-10,1000)],'MAR09034':[(-30, 1000)],'MAR03964_RPE':[(0,0)]})
fba_fva_analysis(mod, Human1_dict, 'MAR03964_PR')

In [ ]:
# set simulation parameters
from src.analysis import create_permutation_dicts

# experimental values in dicts with rxn ID as key and bounds (lb,ub) as values
Human1_dict = {'MAR09048':[(-10,1000)],'MAR09034':[(-30, 1000)],'MAR03964_RPE':[(0,0),(40,40),(80,80)]}
Human1_dict_p = create_permutation_dicts(Human1_dict)

fba_fva_analysis(mod,Human1_dict_p, 'MAR03964_PR')